In [13]:
!pip install requests pandas openpyxl

import requests
import pandas as pd
from datetime import datetime
from IPython.display import display

history = pd.DataFrame(columns=["IP","AbuseIPDB_Status","Confidence_Score","Final_Status","Timestamp"])

ABUSE_API_KEY = "YOUR_ABUSE_KEY"


In [14]:
def check_abuseipdb(ip):
    """
    Check a single IP against AbuseIPDB.
    Returns (status, score).
    """
    url = f"https://api.abuseipdb.com/api/v2/check?ipAddress={ip}&maxAgeInDays=90"
    headers = {"Key": ABUSE_API_KEY, "Accept": "application/json"}
    try:
        res = requests.get(url, headers=headers).json()
        score = res['data']['abuseConfidenceScore']
        status = "Malicious" if score > 50 else "Safe"
        return status, score
    except Exception as e:
        return "Safe", 0

In [16]:
def check_multiple_ips(ip_list):
    global history
    results = []

    for ip in ip_list:
        status, score = check_abuseipdb(ip)
        final_status = status

        new_row = pd.DataFrame([{
            "IP": ip,
            "AbuseIPDB_Status": status,
            "Confidence_Score": score,
            "Final_Status": final_status,
            "Timestamp": datetime.now()
        }])

        history = pd.concat([history, new_row], ignore_index=True)
        results.append(new_row)

    return pd.concat(results, ignore_index=True)


In [17]:
def display_colored(df):
    def color_status(val):
        if val == "Malicious":
            color = 'red'
        elif val == "Safe":
            color = 'green'
        else:
            color = 'black'
        return f'color: {color}; font-weight: bold'

    display(df.style.applymap(color_status, subset=['Final_Status']))



In [18]:
ip_input = input("Enter IPs separated by comma: ")
ip_list = [ip.strip() for ip in ip_input.split(",") if ip.strip()]

results_df = check_multiple_ips(ip_list)
display_colored(results_df)


Enter IPs separated by comma: 8.8.8.8


/tmp/ipython-input-802688719.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  history = pd.concat([history, new_row], ignore_index=True)
/tmp/ipython-input-142407601.py:11: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  display(df.style.applymap(color_status, subset=['Final_Status']))


,IP,AbuseIPDB_Status,Confidence_Score,Final_Status,Timestamp
0,8.8.8.8,Safe,0,Safe,2025-08-31 04:59:16.513174


In [23]:
from google.colab import files
import pandas as pd

print(" Please upload a CSV file with a column named 'IP'")
uploaded = files.upload()

if uploaded:

    filename = list(uploaded.keys())[0]
    print(f" Uploaded file: {filename}")

    try:
        df_csv = pd.read_csv(filename)

        if "IP" not in df_csv.columns:
            print("The CSV must contain a column named 'IP'")
        else:
            ip_list = df_csv["IP"].astype(str).str.strip().tolist()
            results_df = check_multiple_ips(ip_list)
            display(display_colored(results_df))
    except Exception as e:
        print(f"Error reading file: {e}")


 Please upload a CSV file with a column named 'IP'


Saving sample_ips.csv to sample_ips (1).csv
 Uploaded file: sample_ips (1).csv


/tmp/ipython-input-142407601.py:11: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  display(df.style.applymap(color_status, subset=['Final_Status']))


,IP,AbuseIPDB_Status,Confidence_Score,Final_Status,Timestamp
0,8.8.8.8,Safe,0,Safe,2025-08-31 05:04:36.507235
1,8.8.4.4,Safe,0,Safe,2025-08-31 05:04:36.728871
2,1.1.1.1,Safe,0,Safe,2025-08-31 05:04:36.859192
3,9.9.9.9,Safe,0,Safe,2025-08-31 05:04:37.038734
4,192.168.1.1,Safe,0,Safe,2025-08-31 05:04:37.229499
5,10.0.0.1,Safe,0,Safe,2025-08-31 05:04:37.361524
6,172.16.0.1,Safe,0,Safe,2025-08-31 05:04:37.472428
7,185.199.108.153,Safe,0,Safe,2025-08-31 05:04:37.693998
8,45.77.165.194,Safe,0,Safe,2025-08-31 05:04:37.826244
9,185.62.189.27,Safe,0,Safe,2025-08-31 05:04:37.947588


None